In [19]:
import sqlite3


# connect to database
con = sqlite3.connect('C:\\NBA\\nba.db') # create database in the directory where the .py file is saved
cur = con.cursor()
cur2 = con.cursor()

total_spend = 0
projected_score = 0
roster_pos = {}
roster_scores = {}

# find the best value players
    # forwards
cur.execute("""
            SELECT * 
            FROM dk_salaries 
            WHERE position IN ('SF','PF','F')
            ORDER BY adjusted_multiplier DESC
            """)
for row in cur:
    if row[1] not in roster_pos:
        if row[2] not in roster_pos.values():
            if total_spend + row[3] <= 50000:
                roster_pos[row[1]] = row[2]
                roster_scores[row[1]] = row[5]
                total_spend += row[3]
                projected_score += row[5]
            
    # guards
cur.execute("""
            SELECT * 
            FROM dk_salaries 
            WHERE position IN ('PG','G','SG')
            ORDER BY adjusted_multiplier DESC
            """)
for row in cur:
    if row[1] not in roster_pos:
        if row[2] not in roster_pos.values():
            if total_spend + row[3] <= 50000:
                roster_pos[row[1]] = row[2]
                roster_scores[row[1]] = row[5]
                total_spend += row[3]
                projected_score += row[5]

    # centers
cur.execute("""
            SELECT * 
            FROM dk_salaries 
            WHERE position IN ('C')
            ORDER BY adjusted_multiplier DESC
            """)
for row in cur:
    if row[1] not in roster_pos:
        if row[2] not in roster_pos.values():
            if total_spend + row[3] <= 50000:
                roster_pos[row[1]] = row[2]
                roster_scores[row[1]] = row[5]
                total_spend += row[3]
                projected_score += row[5]
                
    # utility
cur.execute("""
            SELECT * 
            FROM dk_salaries 
            WHERE 1=1
            ORDER BY adjusted_multiplier DESC
            """)
for row in cur:
    if row[1] not in roster_pos:
        if total_spend + row[3] <= 50000:
            roster_pos[row[1]] = 'Util'
            roster_scores[row[1]] = row[5]
            total_spend += row[3]
            projected_score += row[5]
            break

# find improvements
while 1==1:
    roster_improvements_pos = {}
    roster_improvements_scores = {}
        # forwards
    cur.execute("""
                SELECT * 
                FROM dk_salaries 
                WHERE position IN ('SF','PF','F') 
                ORDER BY adjusted_multiplier DESC
                """)
    for record in cur:
        # retrieve current salary for position
        pname_strip = roster_pos.keys()[roster_pos.values().index(record[2])]
        cur2.execute("SELECT salary FROM dk_salaries WHERE player_name_strip = '" + pname_strip + "'")
        for p in cur2:
            current_sal = p[0]
        if record[1] not in roster_pos: # player not currently on roster
            if record[1] not in roster_improvements_pos: # player not currently on improvement roster
                if record[2] not in roster_improvements_pos.values(): # position not taken in improvements
                    if record[3] + total_spend - current_sal <= 50000: # can afford player
                        if record[5] > roster_scores[roster_pos.keys()[roster_pos.values().index(record[2])]]: # more projected points
                            roster_improvements_pos[record[1]] = record[2]
                            roster_improvements_scores[record[1]] = record[5] - roster_scores[roster_pos.keys()[roster_pos.values().index(record[2])]]

        # guards
    cur.execute("""
                SELECT * 
                FROM dk_salaries 
                WHERE position IN ('PG','G','SG')
                ORDER BY adjusted_multiplier DESC
                """)
    for record in cur:
        # retrieve current salary for position
        pname_strip = roster_pos.keys()[roster_pos.values().index(record[2])]
        cur2.execute("SELECT salary FROM dk_salaries WHERE player_name_strip = '" + pname_strip + "'")
        for p in cur2:
            current_sal = p[0]
        if record[1] not in roster_pos: # player not currently on roster
            if record[1] not in roster_improvements_pos: # player not currently on improvement roster
                if record[2] not in roster_improvements_pos.values(): # position not taken in improvements
                    if record[3] + total_spend - current_sal <= 50000: # can afford player
                        if record[5] > roster_scores[roster_pos.keys()[roster_pos.values().index(record[2])]]: # more projected points
                            roster_improvements_pos[record[1]] = record[2]
                            roster_improvements_scores[record[1]] = record[5] - roster_scores[roster_pos.keys()[roster_pos.values().index(record[2])]]       

        # centers
    cur.execute("""
                SELECT * 
                FROM dk_salaries 
                WHERE position IN ('C')
                ORDER BY adjusted_multiplier DESC
                """)
    for record in cur:
        # retrieve current salary for position
        pname_strip = roster_pos.keys()[roster_pos.values().index(record[2])]
        cur2.execute("SELECT salary FROM dk_salaries WHERE player_name_strip = '" + pname_strip + "'")
        for p in cur2:
            current_sal = p[0]
        if record[1] not in roster_pos: # player not currently on roster
            if record[1] not in roster_improvements_pos: # player not currently on improvement roster
                if record[2] not in roster_improvements_pos.values(): # position not taken in improvements
                    if record[3] + total_spend - current_sal <= 50000: # can afford player
                        if record[5] > roster_scores[roster_pos.keys()[roster_pos.values().index(record[2])]]: # more projected points
                            roster_improvements_pos[record[1]] = record[2]
                            roster_improvements_scores[record[1]] = record[5] - roster_scores[roster_pos.keys()[roster_pos.values().index(record[2])]]

        # utility
    cur.execute("""
                SELECT * 
                FROM dk_salaries 
                ORDER BY adjusted_multiplier DESC
                """)
    for record in cur:
        # retrieve current salary for position
        pname_strip = roster_pos.keys()[roster_pos.values().index('Util')]
        cur2.execute("SELECT salary FROM dk_salaries WHERE player_name_strip = '" + pname_strip + "'")
        for p in cur2:
            current_sal = p[0]
        if record[1] not in roster_pos: # player not currently on roster
            if record[1] not in roster_improvements_pos: # player not currently on improvement roster
                if record[3] + total_spend - current_sal <= 50000: # can afford player
                    if record[5] > roster_scores[roster_pos.keys()[roster_pos.values().index('Util')]]: # more projected points
                        roster_improvements_pos[record[1]] = 'Util'
                        roster_improvements_scores[record[1]] = record[5] - roster_scores[roster_pos.keys()[roster_pos.values().index('Util')]] 
                        
    # check break condition
    if bool(roster_improvements_pos) == False:
        break
    
    # make roster update
    added_player = roster_improvements_scores.keys()[roster_improvements_scores.values().index(max(roster_improvements_scores.values()))]
    removed_player = roster_pos.keys()[roster_pos.values().index(roster_improvements_pos[roster_improvements_scores.keys()[roster_improvements_scores.values().index(max(roster_improvements_scores.values()))]])]

    del roster_pos[removed_player]
    del roster_scores[removed_player]

    # get added player's salary and projected_dk score
    cur.execute("SELECT salary, projected_dk_score FROM dk_salaries WHERE player_name_strip = '" + added_player + "'")
    for record in cur:
        added_sal = record[0]
        added_score = record[1]

    # get removed player's salary and projected_dk score
    cur.execute("SELECT salary, projected_dk_score FROM dk_salaries WHERE player_name_strip = '" + removed_player + "'")
    for record in cur:
        removed_sal = record[0]
        removed_score = record[1]

    roster_pos[added_player] = roster_improvements_pos[added_player]
    roster_scores[added_player] = added_score

    total_spend = total_spend - removed_sal + added_sal
    projected_score = projected_score - removed_score + added_score

    
